In [ ]:
pip install transformers datasets peft accelerate bitsandbytes

In [1]:
#loading the dataset
from datasets import load_dataset
dataset = load_dataset('sms_spam')
print(dataset['train'][0])

README.md:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

{'sms': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n', 'label': 0}


In [2]:
#tokenization
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["sms"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/5574 [00:00<?, ? examples/s]

In [3]:
print(tokenized_dataset["train"][0])

{'sms': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n', 'label': 0, 'input_ids': [101, 2175, 2127, 18414, 17583, 2391, 1010, 4689, 1012, 1012, 2800, 2069, 1999, 11829, 2483, 1050, 2307, 2088, 2474, 1041, 28305, 1012, 1012, 1012, 25022, 2638, 2045, 2288, 26297, 28194, 1012, 1012, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [6]:
#formating the dataset for finetuning/training
tokenized_dataset.set_format( type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"]
)


In [8]:
#loading the model and peft = lora
from transformers import AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2  # 2 classes for ham and spam
                                                          )

#LoRA Configs
peft_config = LoraConfig(
    r=8,                        
    lora_alpha=16,               
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,   
)

#wrap the model with LoRA 
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.


trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700


In [10]:
#training configs
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./lora_spam_model",  
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
)

In [11]:
#data collator for padding batches
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
#trainer engine
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_21948\867125470.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
#train/finetune the lora model to get the desired result/model
trainer.train()

C:\Users\Admin\miniconda3\envs\llm-env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.516600
20,0.469800
30,0.391900
40,0.444500
50,0.419700
60,0.406500
70,0.347200
80,0.354100
90,0.353700


In [ ]:
#saving the model
trainer.save_model("lora-spam-detector")

In [ ]:
#saving the tokenizer
tokenizer.save_pretrained("lora-spam-detector")

In [ ]:
#loading the model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig


peft_config = PeftConfig.from_pretrained("lora-spam-detector")
base_model = AutoModelForSequenceClassification.from_pretrained(peft_config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, "lora-spam-detector")
tokenizer = AutoTokenizer.from_pretrained("lora-spam-detector")

model.eval()


In [ ]:
#gradio demo
import gradio as gr
import torch

#prediction function
def classify_sms(message):
    inputs = tokenizer(
        message,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128,
    )
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return "🚫 Spam" if predicted_class == 1 else "✅ Ham"

#building the UI
demo = gr.Interface(
    fn=classify_sms,
    inputs=gr.Textbox(label="Enter SMS Message"),
    outputs=gr.Label(label="Prediction"),
    title="📱 SMS Spam Classifier",
    description="Enter an SMS message below and see if it's spam or not, powered by LoRA + BERT."
)

# Launch the app
demo.launch()